# ASG Quant Fund Project

#### Armaan Gandhara

## Data Loader Development

In [16]:
import yfinance as yf
import pandas as pd
from typing import List, Union

In [17]:
class data_loader:

    def __init__(self):
        pass

    def get_data(self, ticker: str, start: str, end: str) -> pd.DataFrame:
        ticker = ticker.replace('.', '-')

        data = yf.download(ticker, start=start, end=end, progress=False)
        if data.empty:
            print(f"[!] Failed to download {ticker}. Skipping.")
            return None
        data.dropna(inplace=True)
        if 'Adj Close' in data.columns:
            data.drop(columns=['Adj Close'], inplace=True)
        #data.rename(columns={'Open': 'open','High': 'high','Low': 'low','Close': 'close','Volume': 'volume'}, inplace=True)

        #required_cols = ['open', 'high', 'low', 'close', 'volume']
        #data = data[required_cols]
        data = data.droplevel('Ticker', axis=1)
        data.reset_index(inplace=True)

        return data


    def get_multiple_data(self, tickers: List[str], start: str, end: str) -> dict:
        data_dict = {}
        for ticker in tickers:
            df = self.get_data(ticker, start, end)
            if df is not None:
                data_dict[ticker] = df

        return data_dict

In [18]:
dt = data_loader()
tsla = dt.get_data("TSLA", '2020-01-01', '2020-01-10')
tsla
tickers = ['AAPL', 'BRK.B', 'JPM']
dater = dt.get_multiple_data(tickers, '2025-01-01', '2025-01-10')
dater['BRK.B']

Price,Date,Close,High,Low,Open,Volume
0,2025-01-02,451.100006,456.890015,450.029999,455.959991,3746400
1,2025-01-03,453.559998,454.529999,450.119995,452.529999,2884600
2,2025-01-06,451.410004,456.239990,450.570007,453.850006,4072900
3,2025-01-07,452.920013,456.519989,451.100006,452.799988,3507200
4,2025-01-08,451.839996,454.000000,449.630005,453.630005,3933300


## Strategy 1 Mean Reversion

### Not Used

In [19]:
import pandas as pd
import pandas_ta as ta

In [ ]:

class mean_reversion_strategy:
    def __init__(self, lookback: int = 20, std_dev: float = 2.0, threshold: float = 0.0):
        self.lookback = lookback
        self.std_dev = std_dev
        self.threshold = threshold

    def generate_signals(self, data: pd.DataFrame) -> pd.DataFrame:
        df = data.copy()

        bb = ta.bbands(close=df['Close'], length=self.lookback, std=self.std_dev)

        if bb is None or bb.empty:
            raise ValueError("Bollinger Bands calculation failed. Check input data.")

        df = df.join(bb)

        df['signal'] = 0  # Default to hold (0)
        
        df.loc[df['Close'] < df[f'BBL_{self.lookback}_{self.std_dev}'] * (1 - self.threshold), 'signal'] = 1
        
        df.loc[df['Close'] > df[f'BBU_{self.lookback}_{self.std_dev}'] * (1 + self.threshold), 'signal'] = -1

        return df[['signal']]


In [48]:
dt = data_loader()
tsla = dt.get_data("TSLA", '2025-01-01', '2025-06-01')
mrs = mean_reversion_strategy()
signals = mrs.generate_signals(tsla)
signals.value_counts()


signal
 0        92
 1         7
-1         3
Name: count, dtype: int64

### Used

In [ ]:
from backtesting import Strategy, Backtest
import pandas as pd
import pandas_ta as pdt

class MeanReversionBacktestWrapper(Strategy):
    def init(self):
        price = pd.Series(self.data.Close)
        bb = pdt.bbands(close=price, length=20, std=2.0)

        self.lower = self.I(lambda: bb[f'BBL_20_2.0'])
        self.upper = self.I(lambda: bb[f'BBU_20_2.0'])

    def next(self):
        price = self.data.Close[-1]
        if price < self.lower[-1] and not self.position:
            self.buy()
        elif price > self.upper[-1] and not self.position:
            self.sell()

        if self.position.is_long and price > self.data.Close[-2]:
            self.position.close()
        elif self.position.is_short and price < self.data.Close[-2]:
            self.position.close()


## Backtesting Engine

In [ ]:
class GenericBacktestEngine:
    def __init__(self, data: pd.DataFrame, strategy_cls, strategy_kwargs: dict = None, cash: float = 10000, commission: float = 0.002):
        self.data = data
        self.strategy_cls = strategy_cls
        self.strategy_kwargs = strategy_kwargs or {}
        self.cash = cash
        self.commission = commission
        

    def run(self):
        bt = Backtest(
            self.data,
            self.strategy_cls,
            cash=self.cash,
            commission=self.commission
        )
        stats = bt.run(**self.strategy_kwargs)
        return stats

    def plot(self):
        bt = Backtest(
            self.data,
            self.strategy_cls,
            cash=self.cash,
            commission=self.commission
        )
        bt.run(**self.strategy_kwargs)
        bt.plot()


In [46]:
dt = data_loader()
tsla = dt.get_data("TSLA", '2020-01-01', '2025-06-01')
bt = Backtest(tsla, MeanReversionBacktestWrapper, cash=10000, commission=0.002)
stats = bt.run()
bt.plot()



C:\Users\agand\AppData\Local\Temp\ipykernel_14384\477152792.py:3: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(tsla, MeanReversionBacktestWrapper, cash=10000, commission=0.002)


GridPlot(id='p2360', ...)

In [30]:
stats

Start                                     0.0
End                                    1359.0
Duration                               1359.0
Exposure Time [%]                    20.66176
Equity Final [$]                   1337.95517
Equity Peak [$]                       10000.0
Commissions [$]                    1405.07313
Return [%]                          -86.62045
Buy & Hold Return [%]               710.98916
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Alpha [%]                           -33.25785
Beta                                 -0.07505
Max. Drawdown [%]                   -87.27737
Avg. Drawdown [%]                   -87.27737
Max. Drawdown Duration                 1339.0
Avg. Drawdown Duration                 1339.0
# Trades                                 80.0
Win Rate [%]                      